# Notebook for sales dashboard

### Data import and pre-processing

In [ ]:
# Importing packages
import pandas as pd
import streamlit as st 
import plotly_express as px


In [ ]:
# Reading the CSV and understanding the basic structure
file_path = '/Users/danfriedman/Dropbox/TripleTen Data Science/Projects/sales-project/2023_sales_data.csv'
df = pd.read_csv(file_path)

df.head(10)

In [ ]:
df.info()

In [ ]:
# convert dates to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.info()

In [ ]:
df.describe()

From the max of the Date column, some records have 2024 as the year. This was an error in generating the file, so I want to remove these rows from the dataframe.

In [ ]:
# extract month and year from datetimes
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df.head()

In [ ]:
twenty_four_records = df[df['Year'] == 2024]
twenty_four_records

In [ ]:
# amend dataframe to remove rows with year of 2024
df = df.drop(twenty_four_records.index)
df.describe()

The max date of 12-31-2023 confirms the drop method worked to remove the year 2024 rows

### Building the streamlit web app

In [ ]:
# Streamlit app title
st.title("Sales Data Dashboard")

#### Setup Framework for Filtering

First we need to set up a category filter

In [ ]:
# Checkbox for filtering by Category
show_category = st.checkbox("Filter by Category")
if show_category:
    category_choice = st.selectbox("Select a category:", df["Category"].unique())
    df = df[df["Category"] == category_choice]

Next set up the month and total year filter

In [ ]:
# Checkbox for enabling Month/Year filter
show_time_filter = st.checkbox("Filter by Month/Year")
if show_time_filter:
    filter_type = st.radio("Select filter type:", ("Month", "Year"))
    if filter_type == "Month":
        month_choice = st.selectbox("Select a month:", sorted(df["Month"].unique()))
        df = df[df["Month"] == month_choice]
    else:
        year_choice = st.selectbox("Select a year:", sorted(df["Year"].unique()))
        df = df[df["Year"] == year_choice]

#### Build the Charts

In [ ]:
# Scatter plot: Total Sales vs Quantity
st.subheader("Total Sales vs Quantity")
fig_scatter = px.scatter(
    df, x="Quantity", y="Total Sales", color="Category", 
    title="Scatter Plot of Total Sales vs Quantity",
    hover_data=["Month"]  # Include Month in hover tooltip
)
st.plotly_chart(fig_scatter)

This is the sales vs quantity scatter plot - comparing revenue of a category (if selected) vs the quantity sold

In [ ]:
# Histogram: Distribution of Total Sales
st.subheader("Distribution of Total Sales")
fig_hist = px.histogram(
    df, x="Total Sales", nbins=30, 
    title="Histogram of Total Sales"
)
st.plotly_chart(fig_hist)

This is the histogram for the distribution of sales

In [ ]:
# Bar Chart: Total Sales for Each Month by Product Category
st.subheader("Product Category Sales")
df_grouped = df.groupby(["Month", "Category"])["Total Sales"].sum().reset_index()
fig_bar = px.bar(
    df_grouped,
    x="Month", y="Total Sales", color="Category",
    title="Total Sales per Month by Product Category",
    barmode="group"
)
st.plotly_chart(fig_bar)

This is the bar chart showing categories sales over the course of each month.